In [1]:
import time
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import os, errno
import glob

In [2]:
source = r"/Users/Mahad/Downloads/drive-download-20200317T004709Z-001/Centroids_Relevant"
outpath = r"/Users/Mahad/Downloads/drive-download-20200317T004709Z-001/Centroids_Trimmed"
window = r"/Users/Mahad/Downloads/drive-download-20200317T004709Z-001/Centroids_Trimmed/Window"

In [3]:
csv_list = glob.glob(source + "//*.csv")

In [4]:
new_list = []
for element in csv_list:
    df = pd.read_csv(source + "//" + element[78:])
    if len(df.X_position.values) < 2000 and not element[78:].startswith("Sham_18") and not element[78:].startswith('Sham_10_0') and not element[78:].startswith('Sham_14_1') and not element[78:].startswith('Sham_17_0'):
        
        new_list.append(element)

In [5]:
start_list = []
end_list = []
name_list = []
for element in new_list:
    df = pd.read_csv(source + "//" + element[78:])
   
    start, end = 0, len(df.X_position.values)
    if np.any(df.X_position.isnull()):
        print(element[78:-4], np.asarray(np.where(df.X_position.isnull())).ravel())
        a = np.asarray(np.where(df.X_position.isnull())).ravel()
        b = a[1:]
        c = a[:-1]
        d = b-c
        if element[78:].startswith("Experimental_1_0"):
            start = 488 
            end = 696
        elif (len(df.X_position.values) - a[-1]) > 40:
            start = a[-1] + 1
            end = len(df.X_position.values)
            
        elif max(d) > 5:
            start = a[np.argmax(d)]+1
            end = a[np.argmax(d)+1]

        
        else:
            if a[0] == 0:
                start = a[-1] + 1
                end = len(df.X_position)

            else:
                start = 0
                end = a[0]

    new_df = df[start:end]
    start_list.append(new_df.FrameNumber.values[0])
    end_list.append(new_df.FrameNumber.values[-1])
    name_list.append(element[78:-4])
    new_df = new_df.reset_index()
    new_df = new_df.drop(columns=['Unnamed: 0', 'index'])
    new_df.to_csv(outpath + "//" + element[78:])

Experimental_7_4_54607 [  0   1   2   3   4 350 351 352 353 354]
Experimental_7_1_36846 [  0   1   2 418 419 420]
Sham_16_9_31431 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  16  17  18
  20  21  22  24  25  26  28  29  30  32  33  34 296 297 298 299 300 301
 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 360 361 362 363 364 365 366 367 368 369 370]
Experimental_7_3_41552 [  0   1   2 353 354 355 356 357]
Experimental_4_1_72340 [0 1 2 3]
Sham_16_8_29633 [1413 1414 1415 1416 1417 1418 1419 1420 1421 1422 1423 1424 1425 1426
 1427 1428 1429]
Sham_7_0_63837 [231 232 233 234 235 236 237 239 240 241 242 243 244 245 246 247 248 249
 250 251 252 253 254 255 256 257 258 259 260 261 262]
Experimental_7_0_36526 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
 274 275 276 277 278 279]
Experimental_1_0_52688 [  0   1 471 472 473 475 476 477 478 479 480 48

In [6]:
df_window  = pd.DataFrame({"start_frame": start_list, "end_frame": end_list, "name": name_list})
df_window = df_window[["start_frame", "end_frame", "name"]]

In [7]:
df_window.to_csv(window + "//filteredFrameWindow.csv")